### Testing Simulation for n = 20 and Normal Base distribution

Here I simulate $N = 100,000$ times from the density with $\sim N(5, 0.1)$ as its base distribution and variance covariance matrix $\Gamma = 0.1 1_n 1_n^t + 0.1 I_n$


In [1]:
using GLMCopula, Random, LinearAlgebra, StatsBase, GLM

Random.seed!(12345)

MersenneTwister(UInt32[0x00003039], Random.DSFMT.DSFMT_state(Int32[-870096391, 1072918504, -1812426662, 1073255081, -733866021, 1073404543, 807620846, 1073368448, 1919433844, 1072852359  …  -362113007, 1073100625, -166402106, 1073460158, -1907020342, 721295190, -750225566, -1300227565, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000

In [2]:
n = 20

mean_normal = 5
sd_normal = 0.1

variance_component_1 = 0.1
variance_component_2 = 0.1
Γ = variance_component_1 * ones(n, n) + variance_component_2 * Matrix(I, n, n)

vecd = Vector{ContinuousUnivariateDistribution}(undef, n)

for i in 1:n
    vecd[i] = Normal(mean_normal, sd_normal)
end

nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ);

In [3]:
# simulate 100,000 samples
nsample = 100000

Y_nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, nsample);

In [4]:
d = Normal()
link = IdentityLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaVCObs{T, D, Link}}(undef, nsample)
for i in 1:nsample
    y = Float64.(Y_nsample[i])
    X = ones(n, 1)
    V = [ones(n, n), Matrix(I, n, n)]
    gcs[i] = GLMCopulaVCObs(y, X, V, d, link)
end
gcm = GLMCopulaVCModel(gcs);

In [5]:
initialize_model!(gcm)
@show gcm.β
@show inv.(gcm.τ)
fill!(gcm.Σ, 1.0)
update_Σ!(gcm)
@show gcm.Σ    
GLMCopula.loglikelihood!(gcm, true, true)

gcm.β = [5.000023244077052]
inv.(gcm.τ) = [0.010675516155015595]
gcm.Σ = [0.08961672635943388, 0.0815221181866539]


1.7098191237787711e6

In [6]:
@time fit2!(gcm, IpoptSolver(print_level = 5, max_iter = 2000, hessian_approximation = "exact"))

gcm.β = [5.000023244077052]
gcm.Σ = [0.08961622620337302, 0.08152111773351266]
gcm.β = [5.000023244077052]
gcm.Σ = [0.08961572674860344, 0.08152011868319653]

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        1

Total number of variables............................:        1
                     variables with only lower bounds

In [7]:
println("estimated mean = $(gcm.β[1]); true mean value= 5")
println("estimated variance (noise) = $(inv.(gcm.τ[1])); true variance value = $(0.1^2)")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2");

estimated mean = 5.000060755620088; true mean value= 5
estimated variance (noise) = 0.010043041850674408; true variance value = 0.010000000000000002
estimated variance component 1 = 0.0892723455286425; true variance component 1 = 0.1
estimated variance component 2 = 0.08082643832044906; true variance component 2 = 0.1


In [8]:
GLMCopula.loglikelihood!(gcm, true, true)

1.7098192176891055e6